In [1]:
%config Completer.use_jedi = False

In [5]:
from bot import NewsBot

news_bot = NewsBot(verbose=False)

In [6]:
news_bot.ask("Acidentes de moto")

2024-01-09 02:21:47.554 | DEBUG    | bot.newsbot:ask:61 - Pergunta original: Acidentes de moto
2024-01-09 02:21:47.556 | DEBUG    | bot.newsbot:ask:62 - Pergunta melhorada: Quais são as notícias recentes sobre acidentes de moto em Poços de Caldas e região?
2024-01-09 02:21:48.198 | DEBUG    | bot.newsbot:ask:65 - Intenção: Consulta de conteúdo


{'response': '{\n    "resposta": "Foram registrados dois acidentes de moto recentemente em Poços de Caldas. Um deles ocorreu na Avenida Alcoa, na região sul da cidade, onde um motociclista ficou ferido após colidir com uma carreta. O outro acidente foi registrado na Avenida Vereador Edmundo Cardillo, no trevo de acesso ao bairro Bianucci, onde uma moto colidiu com um carro que avançou o sinal de pare. Ambos os acidentes resultaram em ferimentos leves nos motociclistas.",\n    "link": "https://ondapocos.com.br/motociclista-fica-ferido-em-acidente-na-regiao-sul-de-pocos-de-caldas/",\n    "data": "2022-05-27",\n    "titulo": "Motociclista fica ferido em acidente na região sul de Poços de Caldas",\n    "autor": "Aline Rodrigues"\n}',
 'execution_id': 'a258583e76be4b2aaf5677a411adbb2c'}